In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math
import statsmodels.tsa.ar_model as ar_model
import scipy.integrate as integrate
from decimal import Decimal

/Users/lianglinqian/anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from statsmodels.tsa.stattools import adfuller

# calculate ar coefficient(C, rho and std)

In [3]:
def coeff_ar(price1,price2):
    coeff_ar = []
    spread = price1 - price2
    spread.dropna(inplace=True)
    spread_lag = price1.shift()-price2.shift()
    ar = ar_model.AR(spread)
    model_ar = ar.fit(method='mle',maxlag=1,trend = 'c')
    coeff_ar = model_ar.params
    #print(residual)
    #coeff_ar.append(np.std(spread))
    coeff_ar[0]='{:.3f}'.format(Decimal(coeff_ar[0]))
    coeff_ar[1]='{:.3f}'.format(Decimal(coeff_ar[1]))
    coeff_ar['sigma']= np.std((spread-coeff_ar[0]-coeff_ar[1]*spread_lag).dropna())
    return coeff_ar

# calculation of K, theta and sigma

In [4]:
def cal_ma_coeff(rho,C,sigma_i,price1,price2):
    coeff=[]
    if C/(1-rho)<0:
        spread = price2 - price1
        spread.dropna(inplace=True)
        spread_lag = (price2-price1).shift()
        coeff_ar_ = coeff_ar(price2,price1)
        C = coeff_ar_[0]
        rho = coeff_ar_[1]
        sigma_i = np.std((spread-C-rho*spread_lag).dropna())
    coeff.append(-np.log(rho))
    coeff.append(C/(1-rho))
    coeff.append(sigma_i*np.sqrt(-2*np.log(rho)/(1-rho**2)))
    return coeff

if theta smaller than zero, the transaction will be reversed.

# calculation of mu_x and mu_y

In [5]:
def cal_mu_x_y(K,sigma,theta,x):
    mu_x_y =[]
    mu_x = np.sqrt(2*K)*(x-theta)/sigma
    mu_y = np.sqrt(2*K)*(-1*theta)/sigma
    mu_x_y.append(mu_x)
    mu_x_y.append(mu_y)
    return mu_x_y

# calculation of lamda, cn and kn

In [6]:
def cal_k(mu_y,n):
    k=[]
    for i in range(1,n):
        ki = i-0.25+mu_y**2/(math.pi**2)+mu_y*np.sqrt(2*(i-0.25+mu_y**2/(2*math.pi**2)))/math.pi
        k.append(ki)
    return k

In [7]:
def cal_c(k,mu_x,mu_y):
    c = []
    for i,ki in enumerate(k):
        ci=((-1)**(i+2))*2*np.sqrt(ki)*np.exp(0.25*(mu_x**2-mu_y**2))*math.cos(mu_x*np.sqrt(2*ki)-math.pi*ki+math.pi*0.25)/((2*ki-0.5)*(math.pi*np.sqrt(ki)-2**(-0.5)*mu_y))
        c.append(ci)
    return c

In [8]:
def cal_lamda(K,k):
    lamda=[]
    for ki in k:
        lamda_i = K*(2*ki-0.5)
        lamda.append(lamda_i)
    return lamda

# calculation of mu_s and sigma_s

In [9]:
def cal_mu_s(x,T,K,theta):
    mu_s = x*np.exp(-K*T)+theta*(1-np.exp(-K*T))
    return mu_s

In [10]:
def cal_sigma_s(sigma,K,T):
    sigma_s = np.sqrt(sigma**2*(1-np.exp(-2*K*T))/(2*K))
    return sigma_s

# distribution of reaching y

In [11]:
def distr_reach_y(y,t):
    mu_s = x*np.exp(-K*t)+theta*(1-np.exp(-K*t))
    sigma_s = np.sqrt(sigma**2*(1-np.exp(-2*K*t))/(2*K))
    distr = 1.0/(np.sqrt(2*math.pi)*sigma_s)
    distr = distr * np.exp(-0.5*(y-mu_s)**2/(sigma_s**2))
    return distr

# return of reaching y at maturity T

In [12]:
def fix_t(y):
    return (x-y)*distr_reach_y(y,T)

# return of failure to concentrate

In [13]:
def integrand1(y,t):
    result = (x-y)*sum(c*lamda*np.exp(-lamda*t))*distr_reach_y(y,T-t)
    return result


# 1 order integrate

In [14]:
def integrate_1(func,a,b,n=100):
    length = b-a
    interval = 1.0/n
    N = length / interval
    N = int(N)
    result = 0
    for i in range(0,N,1):
        result = result + func(a+i*interval)*interval
    return result
    

# 2 order intergrate

In [15]:
def integrate_2(func,td,tu,yd,yu,n=100):
    len_t = tu-td
    len_y = yu-yd
    interval = 1.0/n
    Nt = int(len_t / interval)
    Ny = int(len_y / interval)
    result = 0
    for i in range(0,Ny):
        for t in range(0,Nt):
            result = result + func(yd+i*interval,td+t*interval)*interval*interval
    return result
#part1,err1= integrate.dblquad(integrand1,0,T,lambda x:0, lambda x:np.inf,args=(c,lamda,coeff,x,T))

the integrate of python package cannot proceed the integrate of the distribution function, so i create two according to definition.

# import data

In [17]:
ngf1= pd.read_excel('/Users/lianglinqian/Desktop/ngfutures.xlsx', sheetname='ng1before')
ngf2= pd.read_excel('/Users/lianglinqian/Desktop/ngfutures.xlsx', sheetname='ng2before')
ngf3= pd.read_excel('/Users/lianglinqian/Desktop/ngfutures.xlsx', sheetname='ng3before')
ngf4= pd.read_excel('/Users/lianglinqian/Desktop/ngfutures.xlsx', sheetname='ng4before')
ngf5= pd.read_excel('/Users/lianglinqian/Desktop/ngfutures.xlsx', sheetname='ng5before')
ngf6= pd.read_excel('/Users/lianglinqian/Desktop/ngfutures.xlsx', sheetname='ng6before')

/Users/lianglinqian/anaconda/envs/py36/lib/python3.6/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [24]:
len(ngf1.index)

2000

This just import ng data

# Data Cleaning

In [16]:
ng={}
ng['1']=ngf1
ng['2']=ngf2
ng['3']=ngf3
ng['4']=ngf4
ng['5']=ngf5
ng['6']=ngf6
for i,value in ng.items():
    value['Date']=pd.to_datetime(value['Date'])
    value.set_index('Date',inplace=True)
    value = value.sort_index(ascending=True)

# calculate ar coefficient

In [17]:
ar_coeff_12 = coeff_ar(ngf1['PX_LAST'],ngf2['PX_LAST'])
ar_coeff_13 = coeff_ar(ngf1['PX_LAST'],ngf3['PX_LAST'])
ar_coeff_14 = coeff_ar(ngf1['PX_LAST'],ngf4['PX_LAST'])
ar_coeff_15 = coeff_ar(ngf1['PX_LAST'],ngf5['PX_LAST'])
ar_coeff_16 = coeff_ar(ngf1['PX_LAST'],ngf6['PX_LAST'])
ar_coeff_23 = coeff_ar(ngf2['PX_LAST'],ngf3['PX_LAST'])
ar_coeff_24 = coeff_ar(ngf2['PX_LAST'],ngf4['PX_LAST'])
ar_coeff_25 = coeff_ar(ngf2['PX_LAST'],ngf5['PX_LAST'])
ar_coeff_26 = coeff_ar(ngf2['PX_LAST'],ngf6['PX_LAST'])
ar_coeff_34 = coeff_ar(ngf3['PX_LAST'],ngf4['PX_LAST'])
ar_coeff_35 = coeff_ar(ngf3['PX_LAST'],ngf5['PX_LAST'])
ar_coeff_36 = coeff_ar(ngf3['PX_LAST'],ngf6['PX_LAST'])
ar_coeff_45 = coeff_ar(ngf4['PX_LAST'],ngf5['PX_LAST'])
ar_coeff_46 = coeff_ar(ngf4['PX_LAST'],ngf6['PX_LAST'])
ar_coeff_56 = coeff_ar(ngf5['PX_LAST'],ngf6['PX_LAST'])

/Users/lianglinqian/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/base.py:2248: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.
  self._values[0] < other_diff[0]


In [18]:
name=['1 2','1 3','1 4','1 5','1 6','2 3','2 4','2 5','2 6','3 4','3 5','3 6','4 5','4 6','5 6']
ar_coeff_sum = pd.concat([ar_coeff_12,ar_coeff_13,ar_coeff_14,ar_coeff_15,ar_coeff_16,ar_coeff_23,ar_coeff_24,ar_coeff_25,ar_coeff_26,ar_coeff_34,ar_coeff_35,ar_coeff_36,ar_coeff_45,ar_coeff_46,ar_coeff_56],axis=1)
ar_coeff_sum.columns = name
ar_coeff_sum

,1 2,1 3,1 4,1 5,1 6,2 3,2 4,2 5,2 6,3 4,3 5,3 6,4 5,4 6,5 6
const,-0.007000,-0.006000,-0.004000,-0.004000,-0.004000,-0.002000,-0.002000,-0.002000,-0.002000,-0.001000,-0.001000,-0.001000,-0.001000,-0.001000,-0.000000
L1.PX_LAST,0.956000,0.977000,0.985000,0.987000,0.988000,0.976000,0.985000,0.986000,0.988000,0.980000,0.983000,0.986000,0.976000,0.985000,0.973000
sigma,0.093418,0.134968,0.161999,0.176663,0.187206,0.083782,0.129361,0.159662,0.175045,0.092099,0.133435,0.159263,0.082208,0.119774,0.099689


# calculation of K, theta and sigma

In [41]:
#spread = ngf1['PX_LAST']-ngf2['PX_LAST']
#spread.dropna(inplace=True)
#spread_lag = ngf1['PX_LAST'].shift()-ngf2['PX_LAST'].shift()
#sigma_i1 = np.std((spread-ar_coeff[0]-ar_coeff[1]*spread_lag).dropna())
exp_return=[]
kappa_ng=[]
sigma_ng=[]
for column in ar_coeff_sum:
    pair = column.split(' ')
    price_1 = ng[pair[0]]['PX_LAST']
    price_2 = ng[pair[1]]['PX_LAST']
    ar_coeff = coeff_ar(price_1,price_2)
    coeff = cal_ma_coeff(ar_coeff[1],ar_coeff[0],ar_coeff[2],price_1,price_2)
    K = coeff[0]
    theta = coeff[1]
    sigma = coeff[2]
    kappa_ng.append(K)
    sigma_ng.append(sigma)
    x = 2*coeff[2]+theta
    mu_x_y = cal_mu_x_y(coeff[0],coeff[2],coeff[1],x)
    T=120.0/252
    mu_x = mu_x_y[0]
    mu_y = mu_x_y[1]
    k= cal_k(mu_y,len(ngf1['PX_LAST'])-1)
    c= cal_c(k,mu_x,mu_y)
    lamda = cal_lamda(coeff[0],k)
    k = np.array(k)
    c = np.array(c)
    lamda = np.array(lamda)
    r = x*sum(c-c*np.exp(-lamda*T))+integrate_1(fix_t,0,100)-integrate_2(integrand1,0,T,0,100)
    exp_return.append(r)

/Users/lianglinqian/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/base.py:2248: FutureWarning: In the future, 'NAT < x' and 'x < NAT' will always be False.
  self._values[0] < other_diff[0]


# calculate expected return

In [56]:
ar_coeff_sum.loc['exp_return']=exp_return
ar_coeff_sum.loc['kappa']=kappa_ng
ar_coeff_sum.loc['sigma']=sigma_ng
ng_return = ar_coeff_sum

In [66]:
ng_return.T

,const,L1.PX_LAST,sigma,exp_return,kappa
1 2,-0.007,0.956,0.095525,0.000790,0.044997
1 3,-0.006,0.977,0.136553,0.009153,0.023269
1 4,-0.004,0.985,0.163270,0.007507,0.015114
1 5,-0.004,0.987,0.177819,-0.008190,0.013085
1 6,-0.004,0.988,0.188345,-0.006877,0.012073
2 3,-0.002,0.976,0.084770,0.005045,0.024293
2 4,-0.002,0.985,0.130359,0.003183,0.015114
2 5,-0.002,0.986,0.160752,-0.004669,0.014099
2 6,-0.002,0.988,0.176081,0.011430,0.012073
3 4,-0.001,0.980,0.093011,-0.001620,0.020203


In [4]:
cl1= pd.read_excel('/Users/lianglinqian/Desktop/cl1.xlsx', sheetname='cl12008before')
cl2= pd.read_excel('/Users/lianglinqian/Desktop/cl1.xlsx', sheetname='cl22008before')
cl3= pd.read_excel('/Users/lianglinqian/Desktop/cl1.xlsx', sheetname='cl32008before')
cl4= pd.read_excel('/Users/lianglinqian/Desktop/cl1.xlsx', sheetname='cl42008before')
cl5= pd.read_excel('/Users/lianglinqian/Desktop/cl1.xlsx', sheetname='cl52008before')
cl6= pd.read_excel('/Users/lianglinqian/Desktop/cl1.xlsx', sheetname='cl62008before')

/Users/lianglinqian/anaconda/envs/py36/lib/python3.6/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [5]:
cl={}
cl['1']=cl1
cl['2']=cl2
cl['3']=cl3
cl['4']=cl4
cl['5']=cl5
cl['6']=cl6
for i,value in cl.items():
    value['Date']=pd.to_datetime(value['Date'])
    value.set_index('Date',inplace=True)
    value = value.sort_index(ascending=True)

In [28]:
ar_coeff_12_cl = coeff_ar(cl1['PX_LAST'],cl2['PX_LAST'])
ar_coeff_13_cl = coeff_ar(cl1['PX_LAST'],cl3['PX_LAST'])
ar_coeff_14_cl = coeff_ar(cl1['PX_LAST'],cl4['PX_LAST'])
ar_coeff_15_cl = coeff_ar(cl1['PX_LAST'],cl5['PX_LAST'])
ar_coeff_16_cl = coeff_ar(cl1['PX_LAST'],cl6['PX_LAST'])
ar_coeff_23_cl = coeff_ar(cl2['PX_LAST'],cl3['PX_LAST'])
ar_coeff_24_cl = coeff_ar(cl2['PX_LAST'],cl4['PX_LAST'])
ar_coeff_25_cl = coeff_ar(cl2['PX_LAST'],cl5['PX_LAST'])
ar_coeff_26_cl = coeff_ar(cl2['PX_LAST'],cl6['PX_LAST'])
ar_coeff_34_cl = coeff_ar(cl3['PX_LAST'],cl4['PX_LAST'])
ar_coeff_35_cl = coeff_ar(cl3['PX_LAST'],cl5['PX_LAST'])
ar_coeff_36_cl = coeff_ar(cl3['PX_LAST'],cl6['PX_LAST'])
ar_coeff_45_cl = coeff_ar(cl4['PX_LAST'],cl5['PX_LAST'])
ar_coeff_46_cl = coeff_ar(cl4['PX_LAST'],cl6['PX_LAST'])
ar_coeff_56_cl = coeff_ar(cl5['PX_LAST'],cl6['PX_LAST'])
ar_coeff_sum_cl = pd.concat([ar_coeff_12_cl,ar_coeff_13_cl,ar_coeff_14_cl,ar_coeff_15_cl,ar_coeff_16_cl,ar_coeff_23_cl,ar_coeff_24_cl,ar_coeff_25_cl,ar_coeff_26_cl,ar_coeff_34_cl,ar_coeff_35_cl,ar_coeff_36_cl,ar_coeff_45_cl,ar_coeff_46_cl,ar_coeff_56_cl],axis=1)
ar_coeff_sum_cl.columns = name
ar_coeff_sum_cl

,1 2,1 3,1 4,1 5,1 6,2 3,2 4,2 5,2 6,3 4,3 5,3 6,4 5,4 6,5 6
const,-0.002000,0.000000,0.002000,0.004000,0.00600,0.001000,0.002000,0.004000,0.005000,0.001000,0.003000,0.0040,0.001000,0.003000,0.001000
L1.PX_LAST,0.970000,0.983000,0.987000,0.989000,0.99000,0.988000,0.990000,0.991000,0.992000,0.991000,0.992000,0.9920,0.992000,0.993000,0.992000
sigma,0.206102,0.264765,0.309657,0.347894,0.37677,0.098087,0.156784,0.204576,0.239641,0.067522,0.121858,0.1579,0.063803,0.095508,0.055689


In [45]:
exp_return_cl=[]
kappa_cl=[]
sigma_cl=[]
for column in ar_coeff_sum_cl:
    pair = column.split(' ')
    price_1_cl = cl[pair[0]]['PX_LAST']
    price_2_cl= cl[pair[1]]['PX_LAST']
    ar_coeff_cl = coeff_ar(price_1_cl,price_2_cl)
    coeff_cl = cal_ma_coeff(ar_coeff_cl[1],ar_coeff_cl[0],ar_coeff_cl[2],price_1_cl,price_2_cl)
    K = coeff_cl[0]
    kappa_cl.append(K)
    theta = coeff_cl[1]
    sigma = coeff_cl[2]
    sigma_cl.append(sigma)
    x = 2*coeff_cl[2]+theta
    mu_x_y_cl = cal_mu_x_y(coeff_cl[0],coeff_cl[2],coeff_cl[1],x)
    T=120.0/252
    mu_x_cl = mu_x_y_cl[0]
    mu_y_cl = mu_x_y_cl[1]
    k= cal_k(mu_y_cl,len(cl1['PX_LAST'])-1)
    c= cal_c(k,mu_x_cl,mu_y_cl)
    lamda = cal_lamda(coeff_cl[0],k)
    k = np.array(k)
    c = np.array(c)
    lamda = np.array(lamda)
    r = x*sum(c-c*np.exp(-lamda*T))+integrate_1(fix_t,0,100)-integrate_2(integrand1,0,T,0,100)
    exp_return_cl.append(r)

In [54]:
ar_coeff_sum_cl.loc['exp_return']=exp_return_cl
ar_coeff_sum_cl.loc['kappa']=kappa_cl
ar_coeff_sum_cl.loc['sigma']=sigma_cl
cl_return = ar_coeff_sum_cl
cl_return

,1 2,1 3,1 4,1 5,1 6,2 3,2 4,2 5,2 6,3 4,3 5,3 6,4 5,4 6,5 6
const,-0.002000,0.000000,0.002000,0.004000,0.006000,0.001000,0.002000,0.004000,0.005000,0.001000,0.003000,0.004000,0.001000,0.003000,0.001000
L1.PX_LAST,0.970000,0.983000,0.987000,0.989000,0.990000,0.988000,0.990000,0.991000,0.992000,0.991000,0.992000,0.992000,0.992000,0.993000,0.992000
sigma,0.209236,0.267038,0.311685,0.349820,0.378664,0.098679,0.157572,0.205501,0.240604,0.067827,0.122348,0.158535,0.064059,0.095844,0.055913
exp_return,0.005934,0.008026,-0.014763,0.026774,-0.017928,0.007136,0.007809,0.012845,-0.009354,-0.003778,0.009839,0.012693,-0.004520,0.003196,0.003826
kappa,0.030459,0.017146,0.013085,0.011061,0.010050,0.012073,0.010050,0.009041,0.008032,0.009041,0.008032,0.008032,0.008032,0.007025,0.008032


In [55]:
cl_return.T

,const,L1.PX_LAST,sigma,exp_return,kappa
1 2,-0.002,0.970,0.209236,0.005934,0.030459
1 3,0.000,0.983,0.267038,0.008026,0.017146
1 4,0.002,0.987,0.311685,-0.014763,0.013085
1 5,0.004,0.989,0.349820,0.026774,0.011061
1 6,0.006,0.990,0.378664,-0.017928,0.010050
2 3,0.001,0.988,0.098679,0.007136,0.012073
2 4,0.002,0.990,0.157572,0.007809,0.010050
2 5,0.004,0.991,0.205501,0.012845,0.009041
2 6,0.005,0.992,0.240604,-0.009354,0.008032
3 4,0.001,0.991,0.067827,-0.003778,0.009041


In [3]:
ho1= pd.read_excel('/Users/lianglinqian/Desktop/ho_data.xlsx', sheetname='HO1_00')
ho2= pd.read_excel('/Users/lianglinqian/Desktop/ho_data.xlsx', sheetname='HO2_00')
ho3= pd.read_excel('/Users/lianglinqian/Desktop/ho_data.xlsx', sheetname='HO3_00')
ho4= pd.read_excel('/Users/lianglinqian/Desktop/ho_data.xlsx', sheetname='HO4_00')
ho5= pd.read_excel('/Users/lianglinqian/Desktop/ho_data.xlsx', sheetname='HO5_00')
ho6= pd.read_excel('/Users/lianglinqian/Desktop/ho_data.xlsx', sheetname='HO6_00')

/Users/lianglinqian/anaconda/envs/py36/lib/python3.6/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [6]:
ho={}
ho['1']=ho1
ho['2']=ho2
ho['3']=ho3
ho['4']=ho4
ho['5']=ho5
ho['6']=ho6
for i,value in ho.items():
    value['Date']=pd.to_datetime(value['Date'])
    value.set_index('Date',inplace=True)
    value = value.sort_index(ascending=True)

In [33]:
ar_coeff_12_ho = coeff_ar(ho1['PX_LAST'],ho2['PX_LAST'])
ar_coeff_13_ho = coeff_ar(ho1['PX_LAST'],ho3['PX_LAST'])
ar_coeff_14_ho = coeff_ar(ho1['PX_LAST'],ho4['PX_LAST'])
ar_coeff_15_ho = coeff_ar(ho1['PX_LAST'],ho5['PX_LAST'])
ar_coeff_16_ho = coeff_ar(ho1['PX_LAST'],ho6['PX_LAST'])
ar_coeff_23_ho = coeff_ar(ho2['PX_LAST'],ho3['PX_LAST'])
ar_coeff_24_ho = coeff_ar(ho2['PX_LAST'],ho4['PX_LAST'])
ar_coeff_25_ho = coeff_ar(ho2['PX_LAST'],ho5['PX_LAST'])
ar_coeff_26_ho = coeff_ar(ho2['PX_LAST'],ho6['PX_LAST'])
ar_coeff_34_ho = coeff_ar(ho3['PX_LAST'],ho4['PX_LAST'])
ar_coeff_35_ho = coeff_ar(ho3['PX_LAST'],ho5['PX_LAST'])
ar_coeff_36_ho = coeff_ar(ho3['PX_LAST'],ho6['PX_LAST'])
ar_coeff_45_ho = coeff_ar(ho4['PX_LAST'],ho5['PX_LAST'])
ar_coeff_46_ho = coeff_ar(ho4['PX_LAST'],ho6['PX_LAST'])
ar_coeff_56_ho = coeff_ar(ho5['PX_LAST'],ho6['PX_LAST'])
ar_coeff_sum_ho = pd.concat([ar_coeff_12_ho,ar_coeff_13_ho,ar_coeff_14_ho,ar_coeff_15_ho,ar_coeff_16_ho,ar_coeff_23_ho,ar_coeff_24_ho,ar_coeff_25_ho,ar_coeff_26_ho,ar_coeff_34_ho,ar_coeff_35_ho,ar_coeff_36_ho,ar_coeff_45_ho,ar_coeff_46_ho,ar_coeff_56_ho],axis=1)
ar_coeff_sum_ho.columns = name
ar_coeff_sum_ho

,1 2,1 3,1 4,1 5,1 6,2 3,2 4,2 5,2 6,3 4,3 5,3 6,4 5,4 6,5 6
const,-0.002000,0.003000,0.007000,0.010000,0.013000,0.002000,0.004000,0.006000,0.008000,0.002000,0.004000,0.006000,0.00200,0.004000,0.002000
L1.PX_LAST,0.974000,0.985000,0.987000,0.989000,0.990000,0.987000,0.989000,0.990000,0.991000,0.988000,0.991000,0.992000,0.99100,0.992000,0.991000
sigma,0.688773,0.965075,1.177308,1.310432,1.419375,0.414217,0.705489,0.896774,1.045394,0.354411,0.590766,0.785243,0.29607,0.536292,0.281305


In [48]:
exp_return_ho=[]
kappa_ho=[]
sigma_ho=[]
for column in ar_coeff_sum_ho:
    pair = column.split(' ')
    price_1_ho = ho[pair[0]]['PX_LAST']
    price_2_ho= ho[pair[1]]['PX_LAST']
    ar_coeff_ho = coeff_ar(price_1_ho,price_2_ho)
    coeff_ho = cal_ma_coeff(ar_coeff_ho[1],ar_coeff_ho[0],ar_coeff_ho[2],price_1_ho,price_2_ho)
    K = coeff_ho[0]
    kappa_ho.append(K)
    theta = coeff_ho[1]
    sigma = coeff_ho[2]
    sigma_ho.append(sigma)
    x = 2*coeff_ho[2]+theta
    mu_x_y_ho = cal_mu_x_y(coeff_ho[0],coeff_ho[2],coeff_ho[1],x)
    T=120.0/252
    mu_x_ho = mu_x_y_ho[0]
    mu_y_ho = mu_x_y_ho[1]
    k= cal_k(mu_y_ho,len(ho1['PX_LAST'])-1)
    c= cal_c(k,mu_x_ho,mu_y_ho)
    lamda = cal_lamda(coeff_ho[0],k)
    k = np.array(k)
    c = np.array(c)
    lamda = np.array(lamda)
    r = x*sum(c-c*np.exp(-lamda*T))+integrate_1(fix_t,0,100)-integrate_2(integrand1,0,T,0,100)
    exp_return_ho.append(r)

In [52]:
ar_coeff_sum_ho.loc['exp_return']=exp_return_ho
ar_coeff_sum_ho.loc['kappa']=kappa_ho
ar_coeff_sum_ho.loc['sigma']=sigma_ho
ho_return = ar_coeff_sum_ho
ho_return

,1 2,1 3,1 4,1 5,1 6,2 3,2 4,2 5,2 6,3 4,3 5,3 6,4 5,4 6,5 6
const,-0.002000,0.003000,0.007000,0.010000,0.013000,0.002000,0.004000,0.006000,0.008000,0.002000,0.004000,0.006000,0.002000,0.004000,0.002000
L1.PX_LAST,0.974000,0.985000,0.987000,0.989000,0.990000,0.987000,0.989000,0.990000,0.991000,0.988000,0.991000,0.992000,0.991000,0.992000,0.991000
sigma,0.697865,0.972377,1.185019,1.317685,1.426513,0.416930,0.709395,0.901284,1.050123,0.356552,0.593438,0.788399,0.297410,0.538447,0.282578
exp_return,-0.002701,-0.018949,-0.059789,-0.076128,-0.049796,-0.009891,0.006980,-0.009153,-0.034609,-0.009580,0.016100,0.004097,0.008392,0.010612,0.000912
kappa,0.026344,0.015114,0.013085,0.011061,0.010050,0.013085,0.011061,0.010050,0.009041,0.012073,0.009041,0.008032,0.009041,0.008032,0.009041


In [53]:
ho_return.T

,const,L1.PX_LAST,sigma,exp_return,kappa
1 2,-0.002,0.974,0.697865,-0.002701,0.026344
1 3,0.003,0.985,0.972377,-0.018949,0.015114
1 4,0.007,0.987,1.185019,-0.059789,0.013085
1 5,0.010,0.989,1.317685,-0.076128,0.011061
1 6,0.013,0.990,1.426513,-0.049796,0.010050
2 3,0.002,0.987,0.416930,-0.009891,0.013085
2 4,0.004,0.989,0.709395,0.006980,0.011061
2 5,0.006,0.990,0.901284,-0.009153,0.010050
2 6,0.008,0.991,1.050123,-0.034609,0.009041
3 4,0.002,0.988,0.356552,-0.009580,0.012073
